In [2]:
import json
import string
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import spacy

[nltk_data] Downloading package punkt to /Users/yhkuo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/yhkuo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yhkuo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Preprocessing functions

In [3]:
def tokenize(line):
    punct = set(string.punctuation)
    token = word_tokenize(line)
    token = [''.join(c for c in s if c not in punct) for s in token]
    return token

def removeStopword(tokens):
    stopWords = set(stopwords.words('english'))
    return [ token.lower() for token in tokens if token.lower() not in stopWords ]

def lemmatize(tokens):
    return [ WordNetLemmatizer().lemmatize(token) for token in tokens ]

def preprocess(line):
    token = tokenize(line)
    token = removeStopword(token)
    lemmas = [ c for c in lemmatize(token) if c != '']
    return lemmas

In [4]:
def createFrequencyMatrix(lemmas):
    frequency_matrix = {}
    for i in range(len(lemmas)):
        sen = lemmas[i]
        for word in sen:
            if word in frequency_matrix:
                frequency_matrix[word] += 1
            else:
                frequency_matrix[word] = 1
    return  {k: v for k, v in sorted(frequency_matrix.items(), key=lambda item: item[1], reverse=True)}

In [14]:
dataPath = '../final-project/public/data/news.json'
with open(dataPath) as file:
    data = json.load(file)
    print(len(data))

2871


In [17]:
threshold = 1
for story in data:
    content = story['content']
    lemma = []
    for line in content:
        lemma.append(preprocess(line))
    fq = createFrequencyMatrix(lemma)
    
    wordcloud = []
    for key, value in fq.items():
        if value > 1:
            item = {}
            item['word'] = key
            item['value'] = value
            wordcloud.append(item)
    story['wordcloud'] = wordcloud

In [18]:
with open('../final-project/public/data/wordcloud.json', 'w') as file:
    json.dump(data, file)